<a href="https://colab.research.google.com/github/anmolkhurana490/Indian-Celebrity-Face-Classification/blob/main/Image_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 3s (76.3 kB/s)
Reading package lists... Done


In [ ]:
!apt-get install -y chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [ ]:
import os
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import base64
from io import BytesIO
from PIL import Image
import threading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
options=webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dve-shm-uage')

In [ ]:
allowed_formats=["jpg", "png", "cms"]

In [ ]:
#to get correct url of a image in chrome
def fetch_image_url(driver, image, name, results):
    try:
        image.click()
        #time.sleep(0.5)
    except:
        print("exception", name, len(results["images_url"]))

    links=driver.find_elements(By.CSS_SELECTOR, "img[jsname='kn3ccd']")

    for link in links:
        if not link or not link.get_attribute('src'):
            results["skipped_invalid_url"]+=1
            continue

        elif link.get_attribute('src').startswith('data:image'):
            results["images_encoded"].append(link.get_attribute('src'))

        elif link.get_attribute('src').lower()[-3:] in allowed_formats:
            if link.get_attribute('src') in results["images_url"]:
                results["already_exists"]+=1

            else:
                results["images_url"].append(link.get_attribute('src'))
                #print(link.get_attribute('src'))

        else:
            #print("invalid:", link.get_attribute('src'))
            results["skipped_invalid_url"]+=1

In [ ]:
#to get (num) images urls in chrome of a celebrity
def get_images(name, min, num):
    query=f"{name} images"

    results={
        "images_url": [],
        "images_encoded": [],
        "skipped_invalid_url": 0,
        "already_exists": 0,
        "num_scrolls": 0,
    }

    url=f"https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={name}&oq={name}&gs_l=img"

    driver=webdriver.Chrome(options=options)
    driver.get(url)

    while True:
        img_elements=driver.find_elements(By.CSS_SELECTOR, '.F0uyec img')

        for image in img_elements:
            fetch_image_url(driver, image, name, results)
            if len(results["images_url"])>num:
                return results

        while len(results["images_url"])<min:
            print("scrolling down")
            prevHeight=driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            newHeight=driver.execute_script("return document.body.scrollHeight")
            if prevHeight==newHeight:
                print("scrolling failed")
                #load_more_button=driver.find_element(By.CSS_SELECTOR, ".mye4qd")
                #print(load_more_button)
            else:
                results["num_scrolls"]+=1
                break
        else:
            return results

In [ ]:
#to download image from url
def download_image(url, path):
  image_data=requests.get(url).content
  with open(f"/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images/{path}", "wb") as file:
    file.write(image_data)

In [ ]:
#to save based64 encoded image by decoding it
def decode_image(img, path):
  header, encoded=img.split(",")
  img_bytes=base64.b64decode(encoded)
  buffer=BytesIO(img_bytes)
  image=Image.open(buffer).convert('RGB')
  with open(f"/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images/{path}", "wb") as f:
    image.save(f,"JPEG")

In [ ]:
celebrity_names=["Alia Bhatt", "Amitabh Bachchan", "Anupam Kher", "Deepika Padukone", "Hema Malini", "Rajkumar Rao", "Sachin Tendulkar", "Salman Khan", "Shah Rukh Khan", "Shilpa Shetty"]

In [ ]:
#downloading required images of a celebrity
def download_celebrity_images(name):
  #make directory of the celebrity
  if not os.path.exists(f"/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images/{name}"):
      os.mkdir(f"/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images/{name}")

  if len(os.listdir(f"/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images/{name}"))>105:
      print(name, "images fetching start")
      results=get_images(name, 150, 170)
      #to download image from urls
      print(name, "images downloading start")
      for i, url in enumerate(results["images_url"][1:]):
          try:
              download_image(url, f"{name}/{i}.jpg")
          except:
              print("downloading error")
      print(name, len(results["images_url"]), len(results["images_encoded"]), results["skipped_invalid_url"], results["already_exists"], results["num_scrolls"])
      print(name, len(results["images_url"]), "images download successful")

In [ ]:
download_celebrity_images('Shah Rukh Khan')

In [ ]:
if __name__=="__main__":
 for name in ['Rajkumar Rao']:
  download_celebrity_images(name)

In [ ]:
for name in os.listdir("/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/original images"):
    if name not in celebrity_names:
        print(name)

In [ ]:
!zip -r images.zip /home/images/

	zip warning: name not matched: /home/images/

zip error: Nothing to do! (try: zip -r images.zip . -i /home/images/)
